In [2]:
%pip install selenium
%pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install selenium-wire

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install python-whois

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install setuptools

  Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import whois
import os

In [4]:
opts = Options()
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
url = "https://fmoviefree.sc"
directory = f"outputs/{url[8:]}"
os.makedirs(directory, exist_ok=True)
file_path = f"{directory}/whois.txt"
with open(file_path, "w") as file:
    w = whois.whois(url)
    file.write(f"{w.text}")
driver = webdriver.Chrome(options=opts)
driver.get(url)
home = driver.find_element(By.CSS_SELECTOR, "div.hm-button a")
home.click()
movie = driver.find_element(By.XPATH, "//div[@class='ml-item']/a[@title = 'Guardians of the Galaxy Vol. 3']")
movie_name = movie.get_attribute("title")
movie.click()
linking_site_url = driver.find_element(By.CSS_SELECTOR, "div#mv-info a")
linking_site_link = linking_site_url.get_attribute("href")
linking_site_url.click()
time.sleep(10)
file_path = f"{directory}/library_packages.txt"
with open(file_path, "w") as file:
    #print("linking site url: ", linking_site_link, file = file)
    file.write(f"linking site url: {linking_site_link}\n")
    hosting_site_url = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-embed").get_attribute("src")
    #print("hosting site url: ", hosting_site_url, file = file)
    file.write(f"hosting site url: {hosting_site_url}\n")
    script_tags = driver.find_element(By.TAG_NAME, "body").find_elements(By.CSS_SELECTOR, "script[src]")
    link_tags = driver.find_element(By.TAG_NAME, "head").find_elements(By.CSS_SELECTOR, "link[href]")
    script_head_tags = driver.find_element(By.TAG_NAME, "head").find_elements(By.CSS_SELECTOR, "script[src]")
    for script_tag in script_tags:
        src = script_tag.get_attribute("src")
        file.write(f"{src}\n")
    for link_tag in link_tags:
        href = link_tag.get_attribute("href")
        file.write(f"{href}\n")
    for script_head_tag in script_head_tags:
        src = script_head_tag.get_attribute("src")
        file.write(f"{src}\n")
        
file_path = f"{directory}/network_data.txt"
existing_urls = set()
if os.path.isfile(file_path):
    with open(file_path, "r") as file:
        for line in file:
            url = line.split()[0]
            existing_urls.add(url)
else:
    with open(file_path, "w") as file:
        time.sleep(3)
        iframe = driver.find_element(By.XPATH, "//iframe[@id='iframe-embed']")
        driver.switch_to.frame(iframe)
        driver.find_element(By.XPATH,"//div[@aria-label='Start Playback']").click()
        duration = 600
        start_time = time.time()
        while time.time() - start_time < duration:
            for request in driver.requests:
                if request.response and request.url not in existing_urls:
                    file.write(
                        f"{request.url} {request.response.status_code} {request.response.headers['Content-Type']}\n"
                    )
                    existing_urls.add(request.url)
        driver.quit()

KeyboardInterrupt: 

In [3]:
def write_requests(driver,file_path,existing_urls):
    with open(file_path, "a") as file:
        time.sleep(3)
        for request in driver.requests:
            if request.response and request.url not in existing_urls:
                file.write(
                    f"{request.url} {request.response.status_code} {request.response.headers['Content-Type']}\n"
                )
                print(f"{request.url} {request.response.status_code} {request.response.headers['Content-Type']}")
                existing_urls.add(request.url)
opts = Options()
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
opts.add_argument("--mute-audio")
url = "https://livemoetv.com/manchester-yunajted-chelsi-25-05-2023-pryamaya-translyacziya/"
driver = webdriver.Chrome(options=opts)
driver.get(url)
directory = f"outputs/{url[8:].split('/')[0]}"
os.makedirs(directory, exist_ok=True)
file_path = f"{directory}/whois.txt"
with open(file_path, "w") as file:
    w = whois.whois(url)
    file.write(f"{w.text}")
    print(f"{w.text}")
file_path = f"{directory}/network_data.txt"
existing_urls = set()
if os.path.isfile(file_path):
    with open(file_path, "r") as file:
        for line in file:
            url = line.split()[0]
            existing_urls.add(url)
    write_requests(driver,file_path,existing_urls)
else:
    write_requests(driver,file_path,existing_urls)





   Domain Name: LIVEMOETV.COM
   Registry Domain ID: 2454522847_DOMAIN_COM-VRSN
   Registrar WHOIS Server: whois.reg.com
   Registrar URL: http://www.reg.ru
   Updated Date: 2022-11-11T12:47:37Z
   Creation Date: 2019-11-13T05:16:24Z
   Registry Expiry Date: 2023-11-13T05:16:24Z
   Registrar: REGISTRAR OF DOMAIN NAMES REG.RU LLC
   Registrar IANA ID: 1606
   Registrar Abuse Contact Email: abuse@reg.ru
   Registrar Abuse Contact Phone: +74955801111
   Domain Status: clientTransferProhibited https://icann.org/epp#clientTransferProhibited
   Name Server: LOCHLAN.NS.CLOUDFLARE.COM
   Name Server: OLGA.NS.CLOUDFLARE.COM
   DNSSEC: unsigned
   URL of the ICANN Whois Inaccuracy Complaint Form: https://www.icann.org/wicf/
>>> Last update of whois database: 2023-05-25T06:43:14Z <<<

For more information on Whois status codes, please visit https://icann.org/epp

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the reg

In [4]:
import os
import csv
import uuid

surl = "https://livemoetv.com/manchester-yunajted-chelsi-25-05-2023-pryamaya-translyacziya/"
csv_file_path = "outputs/Intel_Data.csv"
fieldnames = ["domain", "specific_url", "intel_object_id", "intel_object_url", "intel_object", "http_return_code", "service"]
domain = surl[8:].split('/')[0]
specific_url = surl

existing_urls = set()
with open(f"outputs/{domain}/network_data.txt", "r") as file:
    for line in file:
        existing_urls.add(line)
        url = line.split()

file_exists = os.path.isfile(csv_file_path)

with open(csv_file_path, mode="a", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()
    specific_url = surl
    for line in existing_urls:
        url = line.split()
        intel_object_id = uuid.uuid4()
        intel_object_url = url[0]
        if len(url) != 3:
            intel_object = ' '.join(url[2:])
        else:
            intel_object = url[2]
        http_return_code = url[1]
        service = "livestream"
        writer.writerow({"domain":domain, "specific_url": specific_url, "intel_object_id": intel_object_id, "intel_object_url":intel_object_url, "intel_object":intel_object, "http_return_code": http_return_code, "service": service})